<a href="https://colab.research.google.com/github/erickfaria/ibge/blob/main/populacao_ibge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.0 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [2]:
import zipfile
from ftplib import FTP
import datetime
from dateutil.relativedelta import relativedelta
import tqdm
import pandas as pd
import numpy as np

In [3]:
year = datetime.date.today().year - 1

In [4]:
pop = f'estimativa_dou_{year}.xls'

ftp = FTP(f'ftp.ibge.gov.br')
ftp.login()
ftp.cwd(f'Estimativas_de_Populacao/Estimativas_{year}')

'250 Directory successfully changed.'

In [5]:
with open(f'{pop}', 'wb') as fp:
  ftp.retrbinary(f'RETR {pop}', fp.write)

# Limpa o Dado

In [20]:
df = pd.read_excel('/content/estimativa_dou_2021.xls', 'Municípios', skiprows=1,
                   nrows=5570, header=0, 
                   names=['nom_uf', 'cod_uf', 'cod_mun', 'nom_mun','pop_mun'],
                   dtype={'UF':str,	'cod_uf':str,	'cod_mun':str,
                           'nom_mun':str, 'pop_mun':str})

In [21]:
df['cod_mun'] = df['cod_uf'] + df['cod_mun']

df['pop_mun'] = df['pop_mun'].str.replace(r"\(.*\)","", regex=True)
df['pop_mun'] = df['pop_mun'].str.replace(r".","", regex=True)

In [22]:
df = df.astype({'nom_uf': str,
                'cod_uf': 'int32',
                'cod_mun': 'int32',
                'nom_mun': str,
                'pop_mun': 'int32'})

In [23]:
df

,nom_uf,cod_uf,cod_mun,nom_mun,pop_mun
0,RO,11,1100015,Alta Floresta D'Oeste,22516
1,RO,11,1100023,Ariquemes,111148
2,RO,11,1100031,Cabixi,5067
3,RO,11,1100049,Cacoal,86416
4,RO,11,1100056,Cerejeiras,16088
...,...,...,...,...,...
5565,GO,52,5222005,Vianópolis,14088
5566,GO,52,5222054,Vicentinópolis,9002
5567,GO,52,5222203,Vila Boa,6451
5568,GO,52,5222302,Vila Propício,5941


# Salva os Dados

In [24]:
df.to_parquet(f'/content/pop_ibge_{year}.parquet.gzip',
              compression='gzip', index=False)